In [28]:
from numpy import linalg as LA
import matplotlib.pyplot as plt
import sys
import os, shutil                         # For issuing commands to the OS.
from os import listdir
from os.path import isfile, join
import time
import random
from matplotlib.font_manager import FontProperties
import pandas as pd
import csv
from scipy import stats
import matplotlib.pyplot as plt
import datetime as dt
import datetime
from scipy.optimize import curve_fit
from scipy.stats import bernoulli
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
from sklearn.decomposition import PCA
import numpy as np
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
#from sklearn.metrics import confusion_matrixfrom sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import brier_score_loss
from sklearn.preprocessing import PolynomialFeatures
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator
from io import BytesIO
import boto3
import urllib
import netCDF4 as nc
from netCDF4 import MFDataset
import urllib.request
from time import sleep
import requests
from netCDF4 import Dataset
import timeit

#-------------------------------------------------------------------------------------------------------------
# there are 37 pressure height levels
# select pressure height and latitude levels
pr0 =  15 #7 # 400mb
pr1 =  25 # 15
lat0 = 50 # 10 #150  # 5 # input in degrees N, must be more southward than lat1
lat1 = 60 #60 #200# 60  # 90
start = timeit.default_timer()
fps = 3
# directories and create lists of years in strings to be able to read in the files, which contain year in name. 
gafs = 'gafs01051979/gafs197901050003.nc' # 00/12 + 3 6 9 12
ggap = 'ggap01051979/ggap197901050000-c.nc' # 00, 06, 12, 18
dir = '/Volumes/Seagate Backup Plus Drive/meteo-badc/ggap/' # 1979-1993 (1995-2003)
dir1 = '/Volumes/Seagate Backup Plus Drive/meteo-2/ggap/' # 1994-2018
#-------------------------------------------------------------------------------------------------------------
# 1979-1993
year0 = 1979
yrs = [(str(year0+i)+'/') for i in range(15)]
yrs_ = [(year0+i) for i in range(15)]
#print(yrs1)
# 1994-2018, in combination with a different hard disk and path
yrs1 = [(str(year0+i)+'/') for i in range(15,40)]
yrs1_ = [(year0+i) for i in range(15,40)]
#print(yrs1)
# choose the hard disk (the data are distributed over 2 disks)
di = 0
if di == 0:
    dir = dir
    yrs = yrs
    yrs_ = yrs_
    mypath = [dir + ye for ye in yrs]
else:
    dir = dir1
    yrs = yrs1
    yrs_ = yrs1_
    mypath = [dir1 + ye for ye in yrs1]    

In [29]:
# constants needed for arc calculation:
R = 6371 * 1e3  # Earth radius, in m
C = 2 * np.pi * R / 512.  # dr = 2*pi*R*cos(pi*phi/180) would be the whole arc

# VO(t, p, latitude, longitude) (s**-1) 
# T(t, p, latitude, longitude)   (K)
# W(t, p, latitude, longitude)   (Pa s**-1)
# R(t, p, latitude, longitude)   (%)
# CC(t, p, latitude, longitude)  (0 - 1)

# ggap:
# t
# p = 37
# latitude = 256
# longitude = 512
# p = 1000, 975, 950, 925, 900, 875, 850, 825, 800, 775, 750, 700, 650, 600,
#     550, 500, 450, 400, 350, 300, 250, 225, 200, 175, 150, 125, 100, 70, 50,
#     30, 20, 10, 7, 5, 3, 2, 1 ;

# I calculated that 4-8 miles atm pressure height corresponds to 440 - 160 mb, so that we need to take the pressure levels
# p = 400, 350, 300, 250, 200, 175, 150 (roughly), which means p[17:24]. (24 not included.. how is it in Python again?)
# lat [ 89.462944    88.766945    88.06699     87.366035    86.6648  1.7543825    1.0526295    0.35087648  -0.35087648  
# http://www.met.reading.ac.uk/~marc/it/snap/varList/eraVars/  --- list of variables


In [30]:
def files_per_day(dir,yr):   # yr is a single year 
    mypaths = dir + yr
    files_in_year = [f for f in listdir(mypaths) if isfile(join(mypaths, f)) and name_is_good(f)]  
    # lists files from path    
    #dates2 = [dates[d] for d in range(len(dates)) if d % 5 == 0] # take only every 5th day, for plots
    dates = np.unique(np.array([f[4:12] for f in files_in_year]))  #     
    # dates: strings corresponding to indices 4:12 in the filenames ggapYYYYDDHHHH....nc
    return dates,files_in_year # days in one year

In [31]:
def Read_files(mypath,files_for_day):
    w_total = []
    v_total = []
    f_all = []
    for file in files_for_day:  # daily values
        # ---------------------------------------------------------------------------------------------------
        # 1. Read in the (wind) data
        # ----------------------------------------------------------------------------------------------------        
        f = Dataset(mypath + file, 'r')
        f_all.append(f)        
        p = f.variables['p'][:]
        lat = f.variables['latitude'][:]
        lon = f.variables['longitude'][:]
        # vort = f.variables['VO'] # vorticity [1/s]
        v = f.variables['U']  # zonal wind (t,p,lat,lon)
        w = f.variables['V']  # meridional wind
        # time = f.variables['t'][:] # in these files this is only 1 value! (daily)
        v_total.append(v)  # (t,p,lat,lon),(t,p,lat,lon),(t,p,lat,lon),...
        w_total.append(w)
    v_total = np.asarray(v_total)
    v_total = np.sum(v_total, axis=0) / len(files_for_day)
    w_total = np.asarray(w_total)
    w_total = np.sum(w_total, axis=0) / len(files_for_day)
    [f.close() for f in f_all]    
    return v_total,w_total,lat,lon,p

In [32]:
d = np.ones((2,3,2))#.reshape((2,3,2)) 3x2 -> 2x3
a = np.ones((2,3,2))
b = np.array([1,2,3])
k = np.sqrt((d*b[:,None])**2+a**2)
print('d',d,np.shape(d))
print('b',b,np.shape(b))
print('d*b',k)

d [[[1. 1.]
  [1. 1.]
  [1. 1.]]

 [[1. 1.]
  [1. 1.]
  [1. 1.]]] (2, 3, 2)
b [1 2 3] (3,)
d*b [[[1.41421356 1.41421356]
  [2.23606798 2.23606798]
  [3.16227766 3.16227766]]

 [[1.41421356 1.41421356]
  [2.23606798 2.23606798]
  [3.16227766 3.16227766]]]


In [33]:
#-----------------------------------------------------------------------------------------------------------------------
# I. Helping functions (circulation)
#-----------------------------------------------------------------------------------------------------------------------
def lat_to_grid(latNorth,lat):
        # this function makes the conversion from 'latitude North' to the corresponding grid coordinate of the field.
        # latNorth is the latitude North counted from the equator, whereas the files start counting coordinates 
        # from the Nort pole
        grid_length_in_degrees = 180/float(len(lat)) # nb of degrees per latitude grid coordinate 
        grid_coordinate = int(np.round((90-latNorth)/float(grid_length_in_degrees)))# given the latitude in degrees N, what is the 
        return grid_coordinate, grid_length_in_degrees  # coordinate index of the given nc.file?

def Main(files_for_day,mypath,p0, p1, lat0, lat1,calculation_type):
    
        v_total,w_total,lat,lon,p = Read_files(mypath,files_for_day)
        
        l0, grid_length_in_degrees = lat_to_grid(lat0,lat) # 50N --> grid 57
        l1, grid_length_in_degrees = lat_to_grid(lat1,lat) # 60N --> grid 43
    
        if calculation_type == 1:
            # Calculate the circulation along a latitude circle around the globe: Sum_over_longitudes(v)*dl
            cir, lat_selected = circulation(p0, p1, l0, l1, v_total, lat)
            #if plo == True:
            #    plot(timestep, p0, p1, l0, l1, cir, year_n)
            # Calculate maximum circulation in given region for given day
            maxi,pre_init,lat_init = max_circ(cir,p0,l0) # get the corresponding pressure height and latitude of the max circulation
            (pre, latit) = maxi
            # print('shape of circ',np.shape(cir),'cir[',pre_init, lat_init,']', cir[pre, latit], 'p', p[pre_init], ' (mb), lat', lat[lat_init],' (degrees) for file',file,'day ',timestep)
            return cir, maxi, pre_init, lat_init
        
        if calculation_type == 2:
            # for each time step, the 'circulation' is calculated as the sum over longitudes of maximal abs(v) over latitude and pressure height intervals.
            # cir is a scalar in this case
            cir = circulation2(l0, l1, p0,p1,v_total, w_total, lon, grid_length_in_degrees)
            return cir

def circulation2(l0,l1,p0,p1,v,w,lon, grid_length_in_degrees): # scalar value per time
    # for a given pressure height, calculate the absolute value of the 2D velocity vectors for each longitude
    # i = lat index 
    if l0 < l1:
        print('There is something wrong with the latitude calculation',l0,'cannot be smaller than',l1)
    max_v = []
    max_lat = np.zeros(p1-p0)
    v_ = v[0,p0:p1,l1:l0,0:len(lon)] # from l1=60N < l0= 50N to l0=50N  
    w_ = w[0,p0:p1,l1:l0,0:len(lon)]
    #v_ = np.flip(v_, axis=1)  # because later l0is larger than l1
    #w_ = np.flip(w_, axis=1) 
  
    grid_arr = np.array([i for i in range(l1,l0)]) # convert grid coordinates...
    alpha_arr = np.array([(90 - grid_length_in_degrees*grid) for grid in grid_arr])# ..into degrees N
    #print('grid ',grid_arr)
    #print('alpha',alpha_arr)
    #TEST
    # grid  [43 44 45 46 47 48 49 50 51 52 53 54 55 56]
    #alpha [59.765625 59.0625   58.359375 57.65625  56.953125 56.25     55.546875
    # 54.84375  54.140625 53.4375   52.734375 52.03125  51.328125 50.625   ]
    
    
    factor = np.array([(np.cos(alpha/180.))**2 for alpha in alpha_arr]) # ...before calculating the factor
    factor = factor[:,None]    
    velocity_norms = np.sqrt((v_**2)*factor + w_**2)
    v_max_lat = np.max(velocity_norms, axis = 1) # take max along latitude-axis
    v_max_p = np.max(v_max_lat, axis = 0)        # take max along pressure height axis
    #plt.plot(range(len(lon)),v_max_p,linewidth=0.8)
    #plt.savefig("circ_1.png")
    #plt.show()
    v_max_sum = np.sum(v_max_p)                  # sum over longitudes 
    return v_max_sum
    
    # removed for loops -------------------------------------------------------------------   
    #for each longitude, calculate the maximal abs value of velocity # for each longitude, calculate the maximal abs value of velocity            
    #velocity_norms = [np.sqrt((v[0,p,i,j]**2)*(np.cos(i/180.))**2+w[0,p,i,j]**2) for i in range(l0,l1)]
    #for j in range(len(lon)):     
    #  for p in range(p0,p1):
        # is an l1-l0 dim vector    
    #    velocity_norms = [np.sqrt((v[0,p,i,j]**2)*(np.cos(i/180.))**2+w[0,p,i,j]**2) for i in range(l0,l1)] # for each longitude, calculate the maximal abs value of velocity                
    #    max_lat[p-p0] = np.max(velocity_norms) # take maximum over latitudes            
    #  max_p = np.max(max_lat[:])               # take max over pressure heights        
    # max_v.append(max_p)  # get one value per longitude (the max abs value of the velocity over lats and pr heights, for given longitude)                       
    # max_v = np.asarray(max_v)        
    # v_max_sum = np.sum(max_v) # sum the absolute values, one value is returned, for a whole range of latitudes...
    

In [34]:
#a_ = np.zeros(10)
#b_ = a_[2:5]
#print(b_)

In [35]:
def max_circ(cir,p0,l0): # for circulation type 1, 'circulation', lost in this script
    #print(np.shape(cir))
    maxim = np.unravel_index(cir[:,:].argmax(), cir[:,:].shape)
    (pre, latit) = maxim
    pre_init = pre + p0
    lat_init = latit + l0
    return maxim,pre_init,lat_init

def name_is_good(f):
    if f[-3:] == '.nc':
        return True
    else:
        False

In [36]:
calculation_type = 2 # sum over absolute values of v
calculate_circulation = True
print('TEST')
if calculate_circulation == True: 
  for k in range(len(yrs_)): # for each year
  #for k in range(1):  
    mypaths = dir + yrs[k]
    #print(mypaths)
    #d = dates[day] # has 365 inputs because of np.unique only takes sorted unique value...same day :-)
    dates,files_in_year = files_per_day(dir,yrs[k]) # days in one year
    #print('year ', yrs_[k])
    timestep = 0    
    file_name = 'Circ1_'+str(yrs_[k])+'.txt'
    f = open(file_name,'w+')
    circulat = np.zeros(len(dates))
    cir_ = []
    t = 0
    for d in dates:  # all files, sorted according to the date, for a given year. Each file corresponds to a year
        files_for_day = [f for f in files_in_year if f[4:12] == d]  # list of files corresponding to the given day d in the loop

        # circulation version 1
        #cir, max_cir, pre, latit = Main(files_for_day,mypaths,pr0, pr1, lat0, lat1,timestep,False)
        #cir.append(cir) # ,pre.append(), latit
        #f.write('%6.3f\t %d\t %d\t \n' % (cir[max_cir],pre,latit))  # \n.. next line
        #f.write(' '.join(str(x) for x in cir)+'\n')
        
        # circulation version 2
        circulat[timestep] = Main(files_for_day,mypaths,pr0, pr1, lat0, lat1,calculation_type)
        #f.write(' '.join(str(x) for x in circulat)+'\n')
        f.write('%6.3f\n' % circulat[timestep]) 
        #print(timestep, 'cir(t) = ', circulat[timestep],' (m/s)')
        cir_.append(circulat[timestep])
        timestep = timestep + 1

    f.close()
    
    print('year ', yrs_[k], 'mean circulation (m/s)', np.mean(cir_))


TEST
year  1979 mean circulation (m/s) 17725.657318271693
year  1980 mean circulation (m/s) 16975.892376267737
year  1981 mean circulation (m/s) 16710.439262484633
year  1982 mean circulation (m/s) 17754.68723115047
year  1983 mean circulation (m/s) 17030.43385112774
year  1984 mean circulation (m/s) 16966.63582153195
year  1985 mean circulation (m/s) 17353.680423396836
year  1986 mean circulation (m/s) 17363.54084538414
year  1987 mean circulation (m/s) 17338.185165244227
year  1988 mean circulation (m/s) 17214.193366089494
year  1989 mean circulation (m/s) 17808.871215594503
year  1990 mean circulation (m/s) 18338.334983165303
year  1991 mean circulation (m/s) 17375.034423874054
year  1992 mean circulation (m/s) 17390.02107555528
year  1993 mean circulation (m/s) 17014.305217933357


In [ ]:
import ccgfilt_corr2
%matplotlib inline
#dir = '/Volumes/Seagate Backup Plus Drive/meteo-badc/ggap/' # 1979-1993 (1995-2003)
#year_list = [(year0+i) for i in range(40)]

#cutoff = 100# np.array([80,40])

year0=1979

year_list = [(year0+i) for i in range(40)]

#print('analysis for years', year_list)

total_cir = [np.loadtxt('Circ1_'+str(j)+'.txt') for j in year_list] #- result is [nparray(365,3) , nparray(366,3) , ...]
#print('total circulation',np.shape(total_cir))
#print(total_cir[:50])

In [ ]:
%matplotlib inline
#dir = '/Volumes/Seagate Backup Plus Drive/meteo-badc/ggap/' # 1979-1993 (1995-2003)
#year_list = [(year0+i) for i in range(40)]
year0=1979
year_list = [(year0+i) for i in range(40)]
print('analysis for years', year_list)

total_cir = [np.loadtxt('Circ1_'+str(j)+'.txt') for j in year_list] #- result is [nparray(365,3) , nparray(366,3) , ...]
print(np.shape(total_cir))
print('shape',[np.shape(x) for x in total_cir])
print('concatenate circulation')
total_cir = np.concatenate(total_cir,axis=0)
print('shape of concatenated circ',np.shape(total_cir))

time_len = [np.shape(x) for x in total_cir]
print('time',len(time_len))

#a=range(4)
#b=[1+i for i in range(4)]
#plt.figure()
#plt.plot(a,b)

print('plot')
#print('why?')
#print()
plt.figure(figsize=(20,10)) 
plt.plot(range(len(time_len)),total_cir,linewidth=0.8)

#plt.show();




In [ ]:
def plot(timestep,p0,p1,l0, l1,cir,year_n):
    plt.switch_backend('agg')
    cir_min = -1500
    cir_max = 1500
    levels = MaxNLocator(nbins=15).tick_values(cir[:, :].min(), cir[:, :].max())
    # fig, (ax0, ax1) = plt.subplots(nrows=2)
    fig, ax1 = plt.subplots(nrows=1)
    im = plt.imshow(np.transpose(cir[:, :]), extent=(p0 * 10, (p1 - 1) * 10, l1 - 1, l0),
                    interpolation='nearest', cmap=cm.gist_rainbow)
    # ax1.set_xticks(range(len(labels1)), [i for i in labels1])
    # ax = fig.add_subplot(131)
    # mesh = ax1.pcolormesh(data, cmap=cm)
    im.set_clim(cir_min, cir_max)
    fig.colorbar(im, shrink=0.6)
    ax1.set_title('daily wind circulation (' + str(year_n) + '), day %03d ' % (timestep))
    ax1.set_xlabel('pressure height')
    ax1.set_ylabel('latitudes')
    fig.tight_layout()
    print('time step', timestep)
    plt.savefig("_tmp%05d.png" % timestep)

def plot_final(p0,p1,l0,l1,cir,var,color):
    fh = 5  # figheight
    fw = 12.5  # figwidth
    m = 6
    plt.switch_backend('agg')
    fig, ax1 = plt.subplots(nrows=1)
    #adjustFigAspect(fig, aspect=.4)
    fig.set_figheight(fh)
    fig.set_figwidth(fw)
    plt.plot((cir),'o',color,linewidth=1.1)
    ax1.set_title('daily '+str(var)+' for region (p,lat) = (['+str(p0)+','+str(p1)+'],['+str(l0)+','+str(l1)+'])')
    ax1.set_xlabel('year')
    ax1.set_ylabel(''+str(var)+'')
    fig.tight_layout()
    plt.xticks(fontsize=m)
    plt.yticks(fontsize=m)
    ax1.legend(fontsize=m)
    plt.savefig(''+str(var)+'1979-2018.png')

def adjustFigAspect(fig, aspect=1):
    '''
    Adjust the subplot parameters so that the figure has the correct
               aspect ratio.
    '''
    xsize, ysize = fig.get_size_inches()
    minsize = min(xsize, ysize)
    xlim = .4 * minsize / xsize
    ylim = .4 * minsize / ysize
    if aspect < 1:
        ylim *= aspect
    else:
        xlim /= aspect
    fig.subplots_adjust(left=.5 - xlim,
                        right=.5 + xlim,
                        bottom=.5 - ylim,
                        top=.5 + ylim)
#-----------------------------------------------------------------------------------------------------------------------


In [ ]:
movie = False
if movie == True:
 timestep = 0
 for d in dates:  # for those files corresponding to a same day, within each year, list all of them

    print('plot figure for time step',timestep)
    fps = 3                                         # nb of frames per second for the movie

    cir_min = -1500
    cir_max = 1500
    movie_figures = True
    if movie_figures == True:

     plt.switch_backend('agg')
     fig = plt.figure()
     levels = MaxNLocator(nbins=15).tick_values(cir[:, :].min(), cir[:, :].max())
     # pick the desired colormap, sensible levels, and define a normalization
     # instance which takes data values and translates those into levels.
     cmap = plt.get_cmap('PiYG')
     norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)
     # fig, (ax0, ax1) = plt.subplots(nrows=2)
     fig, ax1 = plt.subplots(nrows=1)
     pa = np.array([i for i in range(p0, p1)])  # array indices of pressure height
     x = np.array([p[prs] for prs in pa])  # pressure at index, in mbar
     y = lat  # latitude in degrees
     im = plt.imshow(cir[:, :], extent=(x.min(), x.max(), y.max(), y.min()),
                    interpolation='nearest', cmap=cm.gist_rainbow)
     fig.colorbar(im, shrink=0.4)



     #fig = plt.figure()
     #ax = fig.add_subplot(131)
     #mesh = ax1.pcolormesh(data, cmap=cm)
     im.set_clim(cir_min, cir_max)

     ax1.set_title('daily wind circulation (1979), day %03d ' %timestep)
     fig.tight_layout()
     print('time step',timestep)

     plt.savefig("_tmp%05d.png" % timestep)

     timestep = timestep +1
     #plt.clf()  # Clear the figure to make way for the next image.

     if timestep == 364:
        os.system("rm -f movie.mp4")
        os.system("./ffmpeg -r " + str(fps) + " -b 1800 -i _tmp%05d.png movie.mp4")
        os.system("rm _tmp*.png")

        exit()

